In [25]:
import os
from pathlib import Path
from typing import Dict, List, Optional, Tuple

import cv2
import numpy as np

import sys
def setup_project_path() -> Path:
    current = Path.cwd()
    while current != current.parent and not (current / "fpn").exists():
        current = current.parent
    if not (current / "fpn").exists():
        raise RuntimeError("Could not find project_root containing 'fpn' directory.")
    return current
project_root = setup_project_path()
sys.path.insert(0, str(project_root))


In [26]:
def imread_unicode(path, flags=cv2.IMREAD_GRAYSCALE):
    try:
        path = str(path)
        with open(path, "rb") as f:
            data = f.read()
        arr = np.frombuffer(data, np.uint8)
        img = cv2.imdecode(arr, flags)
        return img
    except Exception as e:
        print("[imread_unicode ERROR]", e, "->", path)
        return None

In [27]:
def load_label_any(path: str | Path) -> np.ndarray:
    path = Path(path)
    if path.suffix.lower() == ".npy":
        a = np.load(str(path))
        if a.dtype != np.uint8:
            a = a.astype(np.uint8)
        return a
    else:
        m = imread_unicode(path, cv2.IMREAD_GRAYSCALE)
        if m is None:
            raise FileNotFoundError(path)
        if m.dtype != np.uint8:
            m = m.astype(np.uint8)
        return m


In [28]:
def find_label_file(label_dir: Path, stem: str, priority=(".npy", ".png")) -> Optional[Path]:
    for ext in priority:
        p = label_dir / f"{stem}{ext}"
        if p.exists():
            return p
    return None

In [29]:
def binarize_label(label_0_3: np.ndarray, k: int) -> np.ndarray:
    return (label_0_3 == k).astype(np.uint8)

def area_score(aS: int, aP: int, eps: float = 1.0) -> float:
    # 안정적인 비율 점수 (0~1)
    if aS == 0 and aP == 0:
        return 1.0
    return float(min(aS, aP) / max(aS, aP))

def centroid_of_mask(mask01: np.ndarray) -> Optional[Tuple[float, float]]:
    ys, xs = np.where(mask01 > 0)
    if len(xs) == 0:
        return None
    return (float(xs.mean()), float(ys.mean()))

def centroid_score(cS, cP, sigma: float = 15.0) -> float:
    # 0~1
    if cS is None and cP is None:
        return 1.0
    if cS is None or cP is None:
        return 0.0
    dx = cS[0] - cP[0]
    dy = cS[1] - cP[1]
    d2 = dx*dx + dy*dy
    return float(np.exp(-d2 / (2.0 * sigma * sigma)))

def dilate_mask(mask01: np.ndarray, r: int = 3) -> np.ndarray:
    if r <= 0:
        return (mask01 > 0).astype(np.uint8)
    k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2*r+1, 2*r+1))
    return cv2.dilate(mask01.astype(np.uint8), k, iterations=1)

def iou(maskA01: np.ndarray, maskB01: np.ndarray) -> float:
    A = (maskA01 > 0)
    B = (maskB01 > 0)
    inter = int(np.logical_and(A, B).sum())
    union = int(np.logical_or(A, B).sum())
    if union == 0:
        return 1.0
    return float(inter / union)

def dilated_iou(seed01: np.ndarray, pred01: np.ndarray, r: int = 3) -> float:
    Sd = dilate_mask(seed01, r=r)
    Pd = dilate_mask(pred01, r=r)
    return iou(Sd, Pd)
def rel_geometry_score(
    cS_a, cS_b, cP_a, cP_b,
    sigma_angle_deg: float = 20.0,
    sigma_len: float = 20.0,
) -> Optional[float]:
    """
    두 중심점쌍 (a->b) 벡터의
    - 각도 차이
    - 길이 차이
    를 0~1로 점수화하여 평균.
    하나라도 None이면 None.
    """
    if (cS_a is None) or (cS_b is None) or (cP_a is None) or (cP_b is None):
        return None

    vS = np.array([cS_b[0]-cS_a[0], cS_b[1]-cS_a[1]], dtype=np.float32)
    vP = np.array([cP_b[0]-cP_a[0], cP_b[1]-cP_a[1]], dtype=np.float32)

    lS = float(np.linalg.norm(vS))
    lP = float(np.linalg.norm(vP))
    if lS < 1e-6 or lP < 1e-6:
        return None

    # angle
    dot = float(np.dot(vS, vP) / (lS * lP))
    dot = max(-1.0, min(1.0, dot))
    ang = float(np.degrees(np.arccos(dot)))  # 0..180

    score_angle = float(np.exp(-(ang * ang) / (2.0 * sigma_angle_deg * sigma_angle_deg)))

    # length
    dl = (lS - lP)
    score_len = float(np.exp(-(dl * dl) / (2.0 * sigma_len * sigma_len)))

    return 0.5 * score_angle + 0.5 * score_len


In [30]:
def evaluate_one_char_mvp(
    seed_label: np.ndarray,     # printed
    pred_label: np.ndarray,     # refined handwriting
    k: int,
    sigma_centroid: float = 15.0,
    dilate_r: int = 3,
) -> Dict:
    S = binarize_label(seed_label, k)
    P = binarize_label(pred_label, k)

    aS = int(S.sum())
    aP = int(P.sum())

    cS = centroid_of_mask(S)
    cP = centroid_of_mask(P)

    return {
        "area_seed": aS,
        "area_pred": aP,
        "area_score": area_score(aS, aP),
        "centroid_seed": cS,  # (x,y) or None
        "centroid_pred": cP,
        "centroid_score": centroid_score(cS, cP, sigma=sigma_centroid),
        "iou_dilated": dilated_iou(S, P, r=dilate_r),
    }

In [31]:
def evaluate_pair_mvp(
    seed_label: np.ndarray,
    pred_label: np.ndarray,
    sigma_centroid: float = 15.0,
    dilate_r: int = 3,
    sigma_angle_deg: float = 20.0,
    sigma_len: float = 20.0,
) -> Dict[int, Dict]:
    """
    반환: {1: {...}, 2: {...}, 3: {...}}
    + 각 dict에 rel_score_12, rel_score_23를 추가(가능할 때만)
    """
    out = {}
    for k in (1, 2, 3):
        out[k] = evaluate_one_char_mvp(seed_label, pred_label, k, sigma_centroid, dilate_r)

    # relative geometry (seed & pred centroids)
    cS1, cS2, cS3 = out[1]["centroid_seed"], out[2]["centroid_seed"], out[3]["centroid_seed"]
    cP1, cP2, cP3 = out[1]["centroid_pred"], out[2]["centroid_pred"], out[3]["centroid_pred"]

    rel12 = rel_geometry_score(cS1, cS2, cP1, cP2, sigma_angle_deg=sigma_angle_deg, sigma_len=sigma_len)
    rel23 = rel_geometry_score(cS2, cS3, cP2, cP3, sigma_angle_deg=sigma_angle_deg, sigma_len=sigma_len)

    # 라벨별 dict에 같이 넣어줌(결합 점수는 만들지 않음)
    for k in (1, 2, 3):
        out[k]["rel_score_12"] = rel12
        out[k]["rel_score_23"] = rel23

    return out


In [32]:
def evaluate_folders_mvp(
    printed_dir: str | Path,
    refined_dir: str | Path,
    exts_priority=(".npy", ".png"),
    limit: Optional[int] = None,

    # metric params
    sigma_centroid: float = 15.0,
    dilate_r: int = 3,
    sigma_angle_deg: float = 20.0,
    sigma_len: float = 20.0,
) -> List[Dict]:
    """
    반환 리스트 원소 예:
    {
      "stem": "xxx",
      "scores": {
        1: {...},
        2: {...},
        3: {...},
      }
    }
    """
    printed_dir = Path(printed_dir)
    refined_dir = Path(refined_dir)

    # printed 기준으로 목록 생성
    stems = []
    for p in printed_dir.iterdir():
        if not p.is_file():
            continue
        if p.suffix.lower() not in [e.lower() for e in exts_priority]:
            continue
        stems.append(p.stem)
    stems = sorted(set(stems))

    if limit is not None:
        stems = stems[: int(limit)]

    results: List[Dict] = []

    for stem in stems:
        p_seed = find_label_file(printed_dir, stem, exts_priority)
        p_pred = find_label_file(refined_dir, stem, exts_priority)
        if (p_seed is None) or (p_pred is None):
            continue

        seed = load_label_any(p_seed)
        pred = load_label_any(p_pred)

        # shape 맞추기: seed 기준으로 pred 리사이즈(필요시)
        if pred.shape != seed.shape:
            H, W = seed.shape[:2]
            pred = cv2.resize(pred, (W, H), interpolation=cv2.INTER_NEAREST)

        scores = evaluate_pair_mvp(
            seed_label=seed,
            pred_label=pred,
            sigma_centroid=sigma_centroid,
            dilate_r=dilate_r,
            sigma_angle_deg=sigma_angle_deg,
            sigma_len=sigma_len,
        )

        results.append({
            "stem": stem,
            "seed_path": str(p_seed),
            "pred_path": str(p_pred),
            "scores": scores,   # 1/2/3별
        })

    return results


In [57]:
from pathlib import Path
from typing import List, Dict

def save_mvp_results_to_txt(
    results: List[Dict],
    out_txt_path: str | Path,
):
    out_txt_path = Path(out_txt_path)
    out_txt_path.parent.mkdir(parents=True, exist_ok=True)

    lines = []
    lines.append("# MVP comparison results (printed vs refined handwriting)")
    lines.append("# Per character, per label (1=초성, 2=중성, 3=종성)")
    lines.append("# Note:")
    lines.append("#  - 종성이 없는 문자는 [종성] 항목이 출력되지 않음")
    lines.append("# Fields:")
    lines.append("#  area_score       : 면적 비율 일치도 (0~1)")
    lines.append("#  centroid_score   : 중심 위치 일치도 (0~1)")
    lines.append("#  iou_dilated      : 팽창 IoU (형태+위치, 0~1)")
    lines.append("#  rel_score_12     : 초-중 상대 구조 점수 (0~1)")
    lines.append("#  rel_score_23     : 중-종 상대 구조 점수 (0~1)")
    lines.append("")

    for r in results:
        stem = r["stem"]
        scores = r["scores"]

        lines.append(f"[CHAR] {stem}")

        # ---------- 초성 ----------
        s1 = scores[1]
        rel12_1 = "None" if s1["rel_score_12"] is None else f"{s1['rel_score_12']:.4f}"

        lines.append(
            f"  [초성] "
            f"area_score={s1['area_score']:.4f}, "
            f"centroid_score={s1['centroid_score']:.4f}, "
            f"iou_dilated={s1['iou_dilated']:.4f}, "
            f"rel_score_12={rel12_1}"
        )

        # ---------- 중성 ----------
        s2 = scores[2]
        rel12_2 = "None" if s2["rel_score_12"] is None else f"{s2['rel_score_12']:.4f}"
        rel23_2 = "None" if s2["rel_score_23"] is None else f"{s2['rel_score_23']:.4f}"

        lines.append(
            f"  [중성] "
            f"area_score={s2['area_score']:.4f}, "
            f"centroid_score={s2['centroid_score']:.4f}, "
            f"iou_dilated={s2['iou_dilated']:.4f}, "
            f"rel_score_12={rel12_2}, "
            f"rel_score_23={rel23_2}"
        )

        # ---------- 종성 (있는 경우에만) ----------
        s3 = scores[3]
        if s3["rel_score_23"] is not None:
            rel23_3 = f"{s3['rel_score_23']:.4f}"

            lines.append(
                f"  [종성] "
                f"area_score={s3['area_score']:.4f}, "
                f"centroid_score={s3['centroid_score']:.4f}, "
                f"iou_dilated={s3['iou_dilated']:.4f}, "
                f"rel_score_23={rel23_3}"
            )

        lines.append("")

    out_txt_path.write_text("\n".join(lines), encoding="utf-8")
    print(f"[SAVE] MVP results -> {out_txt_path}")


In [63]:
PRINTED_LABEL_DIR = project_root / "results" / "segment_results" / "printed_chars" / "labels_npy"   # 본인 경로로
REFINED_LABEL_DIR = project_root / "results" / "segment_results" / "suppressed" / "labels_npy"      # 본인 경로로
OUT_TXT = project_root / "results" / "jamo.txt"

results = evaluate_folders_mvp(
    printed_dir=PRINTED_LABEL_DIR,
    refined_dir=REFINED_LABEL_DIR,
    exts_priority=(".npy", ".png"),
    limit=None,

    sigma_centroid=15.0,
    dilate_r=3,
    sigma_angle_deg=20.0,
    sigma_len=20.0,
)
save_mvp_results_to_txt(results, OUT_TXT)

# 문자별 초/중/종 점수만 출력(결합 없음)
for r in results:
    print("stem:", r["stem"])

    scores = r["scores"]

    # 초성
    s1 = scores[1]
    print(
        f"  초성: area={s1['area_score']:.3f}, "
        f"centroid={s1['centroid_score']:.3f}, "
        f"iou_dil={s1['iou_dilated']:.3f}, "
        f"rel12={None if s1['rel_score_12'] is None else round(s1['rel_score_12'], 3)}"
    )

    # 중성
    s2 = scores[2]
    print(
        f"  중성: area={s2['area_score']:.3f}, "
        f"centroid={s2['centroid_score']:.3f}, "
        f"iou_dil={s2['iou_dilated']:.3f}, "
        f"rel12={None if s2['rel_score_12'] is None else round(s2['rel_score_12'], 3)}, "
        f"rel23={None if s2['rel_score_23'] is None else round(s2['rel_score_23'], 3)}"
    )

    # 종성 (있는 경우에만)
    s3 = scores[3]
    if s3["rel_score_23"] is not None:
        print(
            f"  종성: area={s3['area_score']:.3f}, "
            f"centroid={s3['centroid_score']:.3f}, "
            f"iou_dil={s3['iou_dilated']:.3f}, "
            f"rel23={round(s3['rel_score_23'], 3)}"
        )

    print()  # blank line


[SAVE] MVP results -> D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\results\jamo.txt
stem: 00_소_horizontal_no_jong
  초성: area=0.860, centroid=0.242, iou_dil=0.048, rel12=0.726
  중성: area=0.927, centroid=0.979, iou_dil=0.411, rel12=0.726, rel23=None

stem: 01_프_horizontal_no_jong
  초성: area=0.776, centroid=0.995, iou_dil=0.232, rel12=0.936
  중성: area=0.736, centroid=0.819, iou_dil=0.156, rel12=0.936, rel23=None

stem: 02_트_horizontal_no_jong
  초성: area=0.465, centroid=0.715, iou_dil=0.368, rel12=0.956
  중성: area=0.580, centroid=0.823, iou_dil=0.635, rel12=0.956, rel23=None

stem: 03_웨_complex_no_jong
  초성: area=0.890, centroid=0.790, iou_dil=0.252, rel12=0.754
  중성: area=0.796, centroid=0.325, iou_dil=0.105, rel12=0.754, rel23=None

stem: 04_어_vertical_no_jong
  초성: area=0.257, centroid=0.975, iou_dil=0.049, rel12=0.695
  중성: area=0.869, centroid=0.253, iou_dil=0.103, rel12=0.695, rel23=None

stem: 05_분_horizontal_jong
  초성: area=0.671, centroid=0.619, iou_dil=0.267, rel12=0.825
